# Test Model Deployment

## Load Data for Testing

In [ ]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "" # <- needs to be the subscription with the Quick-Starts resource group

#Provide values for the existing Resource Group 
resource_group = "MCW-MLOps-XXXXX" # <- replace XXXXX with your unique identifier

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "quick-starts-ws-XXXXX" # <- replace XXXXX with your unique identifier
workspace_region = "eastus" # <- region of your Quick-Starts resource group

In [ ]:
from azureml.core import Workspace
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

In [ ]:
import numpy as np
import pandas as pd
from azureml.core.dataset import Dataset
from azureml.core import Workspace

# Load the car components labeled data from registered dataset
cardata_ds_name = 'connected_car_components'
cardata_ds = Dataset.get_by_name(workspace=ws, name=cardata_ds_name)

car_components_df = cardata_ds.to_pandas_dataframe()
data = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()
data = np.asarray(data)
labels = np.asarray(labels)

indices = np.arange(data.shape[0])                     
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

### Prepare Test Data

In [ ]:
test_size = 10

x_test = data[0:test_size]
y_test = labels[0:test_size]

print('Shape of test data tensor:', x_test.shape)
print('Shape of test label tensor:', y_test.shape)

## Test Deployment

Provide the **Scoring URI** for the deployed ACI webservice

In [ ]:
# provide the scoring URI
web_service_url = ''

### Make HTTP calls to test the deployed Webservice

In [ ]:
import requests
import json

headers = {'Content-Type':'application/json'}
response = requests.post(web_service_url, json.dumps(x_test.tolist()), headers=headers)
print('Predictions')
print(response.text)
print('Labels')
print(y_test)

# Use built-in Jupyter renderer for the Datasheet based on custom tags

In [ ]:
from IPython.core.display import display, Markdown
from markdown import markdown

def get_tag(tags, tagname):
    text = ''
    try:
        text = tags[tagname]
    except:
        print('Missing tag ' + tagname)
    finally:
        return text

    return text

def get_datasheet(tags):
    
    title = get_tag(tags, 'title')
    description = get_tag(tags, 'datasheet_description')
    details = get_tag(tags, 'details')
    date = get_tag(tags, 'date')
    modeltype = get_tag(tags, 'type')
    version = get_tag(tags, 'version')
    buildnumber = get_tag(tags, 'build_number')
    runid = get_tag(tags, 'run_id')
    helpresources = get_tag(tags, 'help')
    helpurl = get_tag(tags, 'help_url')
    usecase_primary = get_tag(tags, 'usecase_primary')
    usecase_secondary = get_tag(tags, 'usecase_secondary')
    usecase_outofscope = get_tag(tags, 'usecase_outofscope')
    dataset_description = get_tag(tags, 'dataset_description')
    motivation = get_tag(tags, 'motivation')
    caveats = get_tag(tags, 'caveats')

    datasheet = ''
    datasheet+=markdown(f'# {title} \n {description} \n')
    datasheet+=markdown(f'## Model Details \n {details} \n')
    datasheet+=markdown(f'### Model date \n {date} \n')
    datasheet+=markdown(f'### Model type \n {modeltype} \n')
    datasheet+=markdown(f'### Model version \n {version} \n')
    datasheet+=markdown(f'### Release build number \n {buildnumber} \n')
    datasheet+=markdown(f'### Training run id \n {runid} \n')
    datasheet+=markdown(f'### Where to send questions or comments about the model \n Please send questions or concerns using [{helpresources}]({helpurl}) \n')
    datasheet+=markdown('## Intended Uses:\n')
    datasheet+=markdown(f'### Primary use case \n {usecase_primary} \n')
    datasheet+=markdown(f'### Secondary use case \n {usecase_secondary} \n')
    datasheet+=markdown(f'### Out of scope \n {usecase_outofscope} \n')
    datasheet+=markdown('## Evaluation Data:\n')
    datasheet+=markdown(f'### Datasets \n {dataset_description} \n')
    datasheet+=markdown(f'### Motivation \n {motivation} \n')
    datasheet+=markdown(f'### Caveats \n {caveats} \n')

    return datasheet

## Get the deployed Model

In [ ]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.webservice import Webservice

model_name = 'compliance-classifier'
model_version = None

ws = Workspace.from_config()
ws_list = Webservice.list(ws, model_name = model_name)

webservice = None
if(len(ws_list) > 0):
    webservice = ws_list[0]

if model_version != None:
    model = Model(ws, name=model_name, version=model_version)
    print('Model version:', model.version)
else:
    model = Model(ws, name=model_name)
    print('Model version:', model.version)

## Display the Datasheet for deployed Model

In [ ]:
display(Markdown(get_datasheet(model.tags)))